In [1]:
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [2]:
df = spark.read.load("../data/rhci-moby.parquet")

In [3]:
from pyspark.sql.functions import *
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [4]:
pro = df.filter(df["message"].isNotNull()).select(df["ci_job.build_id"], when(df["ci_job.status"] == "SUCCESS", 0).otherwise(1).alias("status"), lower(df["message"]).alias("message"))

In [5]:
tokenizer = Tokenizer().setInputCol("message").setOutputCol("rawTokens")
remover = StopWordsRemover().setInputCol("rawTokens").setOutputCol("tokens")

In [6]:
htf = HashingTF(numFeatures=128, inputCol="tokens", outputCol="features")

hashed = htf.transform(remover.transform(tokenizer.transform(pro)))

In [7]:
hashed.show()

+--------+------+--------------------+--------------------+--------------------+--------------------+
|build_id|status|             message|           rawTokens|              tokens|            features|
+--------+------+--------------------+--------------------+--------------------+--------------------+
|     429|     0|started by remote...|[started, by, rem...|[started, remote,...|(128,[7,24,49,110...|
|     429|     0|wiping out worksp...|[wiping, out, wor...|[wiping, workspac...|(128,[12,41,127],...|
|     429|     0|cloning repositor...|[cloning, reposit...|[cloning, reposit...|(128,[40,100,126]...|
|     429|     0|cloning repositor...|[cloning, reposit...|[cloning, reposit...|(128,[40,100,126]...|
|     429|     0|cloning repositor...|[cloning, reposit...|[cloning, reposit...|(128,[40,100,126]...|
|     429|     0| > git init /var/...|[, >, git, init, ...|[, >, git, init, ...|(128,[1,5,25,28,5...|
|     429|     0| > git --version ...|[, >, git, --vers...|[, >, git, --vers...|(1

In [7]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol="status", featuresCol="features")
model = rf.fit(hashed.sample(True, 0.25).select("status", "features"))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/wibenton/local/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/home/wibenton/local/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43209)
Traceback (most recent call last):
  File "/home/wibenton/local/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/home/wibenton/anaconda/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error

Py4JError: An error occurred while calling o89.fit

In [ ]:
model